In [13]:
import os
import pandas as pd

def extract_folder_info(folder_name):
    """Extract TIME and INSTANCE from the folder name."""
    try:
        first_underscore = folder_name.index('_')
        last_underscore = folder_name.rindex('_')
        TIME = folder_name[:first_underscore]
        INSTANCE = folder_name[first_underscore + 1:last_underscore]
        return TIME, INSTANCE
    except ValueError:
        return None, None

def extract_gap(result_file_path):
    """Extract GAP value from results.txt file."""
    GAP = -1.2345
    if os.path.exists(result_file_path):
        try:
            with open(result_file_path, 'r') as result_file:
                for line in result_file:
                    if line.startswith("GapClosed:"):
                        GAP = float(line.split(':')[1].strip())
                        break
        except Exception as e:
            print(f"Error reading {result_file_path}: {e}")
    return GAP

def extract_solving_time(solving_stat_file_path):
    """Extract SOLVING_TIME value from solving_statistic.txt file."""
    SOLVING_TIME = 3700
    if os.path.exists(solving_stat_file_path):
        try:
            with open(solving_stat_file_path, 'r') as solving_stat_file:
                for line in solving_stat_file:
                    if line.startswith("Total Time"):
                        SOLVING_TIME = float(line.split(':')[1].strip())
                        break
        except Exception as e:
            print(f"Error reading {solving_stat_file_path}: {e}")
    return SOLVING_TIME

def extract_feasibility(result_file_path):
    """Extract FEASIBLE value from results.txt file."""
    FEASIBLE = False
    if os.path.exists(result_file_path):
        try:
            with open(result_file_path, 'r') as result_file:
                for line in result_file:
                    if line.startswith("DbgSolFeasible:"):
                        FEASIBLE = line.split(':')[1].strip().lower() == 'true'
                        break
        except Exception as e:
            print(f"Error reading {result_file_path}: {e}")
    return FEASIBLE

def process_folder(folder_path, folder_name, mode):
    """Process each folder to extract required information."""
    TIME, INSTANCE = extract_folder_info(folder_name)
    if TIME is None or INSTANCE is None:
        return None

    result_file_path = os.path.join(folder_path, 'results.txt')
    solving_stat_file_path = os.path.join(folder_path, 'solving_statistic.txt')

    GAP = extract_gap(result_file_path)
    SOLVING_TIME = extract_solving_time(solving_stat_file_path)
    FEASIBLE = extract_feasibility(result_file_path)

    return [INSTANCE, mode, GAP, SOLVING_TIME, FEASIBLE]

def fetchresult(main_dir):
    """Main function to iterate through directories and collect data."""
    data = []

    for mode in os.listdir(main_dir):
        mode_path = os.path.join(main_dir, mode)
        if os.path.isdir(mode_path):
            for folder_name in os.listdir(mode_path):
                folder_path = os.path.join(mode_path, folder_name)
                if os.path.isdir(folder_path):
                    folder_data = process_folder(folder_path, folder_name, mode)
                    if folder_data:
                        data.append(folder_data)

    df = pd.DataFrame(data, columns=['INSTANCE', 'MODE', 'GAP', 'TIME', 'FEASIBLE'])
    df['GAP'] = df['GAP'].astype(float)

    return df
    # Find and print pairs that a

In [14]:
main_dir = os.path.join(os.getcwd(), 'Data')
df = fetchresult(main_dir)

In [15]:
df[['INSTANCE', 'MODE']].drop_duplicates()

,INSTANCE,MODE
0,traininstance6,vpcnonbasicprojection
1,neos5,vpcnonbasicprojection
2,ran14x18-disj-8,vpcnonbasicprojection
3,30n20b8,vpcnonbasicprojection
4,seymour1,vpcnonbasicprojection
...,...,...
163,30n20b8,gomory
164,gen-ip002,gomory
165,neos5,gomory
166,neos-3381206-awhea,gomory


In [16]:
grouped_df = df.groupby(['INSTANCE', 'MODE']).size().reset_index(name='count')

# Filter pairs that appear more than once
result = grouped_df[grouped_df['count'] > 1]
result

,INSTANCE,MODE,count


In [17]:
# Pivot the DataFrame to the desired format
df_pivot = df.pivot(index='INSTANCE', columns='MODE', values=['GAP', 'TIME','FEASIBLE'])

# Flatten the multi-level columns
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]

gap_columns = [col for col in df_pivot.columns if 'GAP' in col]
print(gap_columns)
df_pivot[gap_columns] = (df_pivot[gap_columns].astype(float)*100).round(2)

['GAP_gomory', 'GAP_intersection', 'GAP_vpcnonbasicprojection', 'GAP_vpctrivialprojection']


In [18]:
df_pivot

,GAP_gomory,GAP_intersection,GAP_vpcnonbasicprojection,GAP_vpctrivialprojection,TIME_gomory,TIME_intersection,TIME_vpcnonbasicprojection,TIME_vpctrivialprojection,FEASIBLE_gomory,FEASIBLE_intersection,FEASIBLE_vpcnonbasicprojection,FEASIBLE_vpctrivialprojection
INSTANCE,,,,,,,,,,,,
30n20b8,9.06,0.00,8.62,-123.45,23.09,97.77,131.77,3700.0,False,False,True,False
beasleyC3,8.05,0.00,0.66,0.66,0.08,7.72,12.75,18.38,False,False,True,True
binkar10_1,7.32,0.00,6.15,6.15,0.1,2.93,6.8,13.24,False,False,True,True
bnatt400,0.00,0.00,0.00,-123.45,2.59,54.22,79.74,3700.0,False,False,True,False
cod105,0.89,0.44,0.44,0.44,6.31,7.88,11.98,170.09,False,False,True,True
cost266-UUE,23.54,0.00,21.18,21.18,0.59,14.32,17.86,23.33,False,False,True,True
csched007,8.68,0.00,3.39,3.39,0.21,12.91,18.21,220.4,False,False,True,True
exp-1-500-5-5,28.40,0.00,20.27,20.27,0.04,5.42,6.3,9.74,False,False,True,True
gen-ip002,2.15,0.91,0.91,0.91,0.01,1.35,3.27,3.32,False,False,True,True


In [ ]:
df_pivot.to_csv("results.csv")

In [ ]:
df_rounded